In [1]:
## `wa`# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [2]:
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from skimage.transform import resize

In [3]:
import os
import numpy as np
import pandas as pd
import time

In [4]:
os.chdir('/kaggle/')

In [5]:
df = pd.read_csv('input/petfinder-pawpularity-score/train.csv')

In [6]:
df_test = pd.read_csv("input/petfinder-pawpularity-score/test.csv")

In [7]:
'''
df_2 = df[(df['Pawpularity']<100) | (df['Pawpularity'] > 3)]
df_2.reset_index
df_2.head()
'''

"\ndf_2 = df[(df['Pawpularity']<100) | (df['Pawpularity'] > 3)]\ndf_2.reset_index\ndf_2.head()\n"

In [8]:
df_2 = df.sample(5000)
df_2.reset_index(inplace = True)
df_2.head()

,index,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,8014,cf8e3efbdf4efe64d3efd4ed1647c46a,0,1,1,1,0,0,0,0,0,0,0,0,17
1,1735,2c26e832aad28adead9e542ca0c8eb1f,0,1,1,1,0,0,0,0,0,0,0,0,25
2,5944,9a0238499efb15551f06ad583a6fa951,0,1,1,1,0,0,0,0,0,0,0,0,21
3,1067,1b1c7f3833828de3c93a5c938cdf602a,0,1,1,1,0,0,0,0,1,1,0,0,36
4,1929,3161b55e50dc9a9b45bddbd36598bdfe,0,0,1,1,0,0,0,0,0,0,0,0,23


In [9]:
'''df_2 = df_2[(df_2['Pawpularity']<100) | (df_2['Pawpularity'] > 2)]
df_2.reset_index
df_2.head()'''

"df_2 = df_2[(df_2['Pawpularity']<100) | (df_2['Pawpularity'] > 2)]\ndf_2.reset_index\ndf_2.head()"

In [10]:
image_size = (96, 96)

In [11]:
%%time
images=[]
loc = 'input/petfinder-pawpularity-score/train/'
for index in range(df_2.shape[0]):
    id=df_2.loc[index,'Id']
    path= loc + str(id) + '.jpg'
    im_array=cv2.imread(path)
    im_array=resize(im_array,image_size,anti_aliasing=True)
    images.append(im_array)

print('Done')

Done
CPU times: user 16min 40s, sys: 6min 19s, total: 22min 59s
Wall time: 13min 49s


In [12]:
X=np.array(images)
X=X.reshape(-1,96,96,3)

del images
#X.shape

In [13]:
y=df_2['Pawpularity']
#y.shape

In [14]:
model=tf.keras.Sequential()

model.add(keras.layers.Conv2D(64, (3,3), input_shape=(96,96, 3), activation='relu', padding='same')) #was 32 instead of 64
#model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Dropout(0.1)) 
#model.add(keras.layers.BatchNormalization())

'''
model.add(keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())
'''
  
model.add(keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
#model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Dropout(0.1))
#model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.2)) #added this
#model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
#model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
#model.add(keras.layers.Dropout(0.2))
#model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Dense(1, activation='linear'))

model.compile(loss='mean_squared_error',optimizer='adam')

print(model.summary())

2022-01-12 18:11:24.481332: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 64)        1792      
_________________________________________________________________
dropout (Dropout)            (None, 96, 96, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 128)       73856     
_________________________________________________________________
dropout_1 (Dropout)          (None, 96, 96, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 1179648)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1179648)           0         
_________________________________________________________________
dense (Dense)                (None, 512)               6

2022-01-12 18:11:24.579734: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-12 18:11:24.580410: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-12 18:11:24.581569: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-12 18:11:24.582721: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [15]:
%%time
model.fit(X,y,epochs=50)

2022-01-12 18:11:27.930440: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-01-12 18:11:29.231091: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


157/157 [==============================] - 21s 93ms/step - loss: 590.0393
Epoch 2/50
157/157 [==============================] - 14s 92ms/step - loss: 449.9427
Epoch 3/50
157/157 [==============================] - 14s 92ms/step - loss: 464.4703
Epoch 4/50
157/157 [==============================] - 15s 93ms/step - loss: 433.3394
Epoch 5/50
157/157 [==============================] - 14s 92ms/step - loss: 373.5955
Epoch 6/50
157/157 [==============================] - 14s 92ms/step - loss: 285.0076
Epoch 7/50
157/157 [==============================] - 14s 92ms/step - loss: 197.7435
Epoch 8/50
157/157 [==============================] - 14s 92ms/step - loss: 127.4534
Epoch 9/50
157/157 [==============================] - 14s 92ms/step - loss: 85.9923
Epoch 10/50
157/157 [==============================] - 14s 92ms/step - loss: 60.1018
Epoch 11/50
157/157 [==============================] - 14s 92ms/step - loss: 40.5680
Epoch 12/50
157/157 [==============================] - 14s 92ms/step - loss: 

In [16]:
images=[]
locT = 'input/petfinder-pawpularity-score/test/'
for index in range(df_test.shape[0]):
    id=df_test.loc[index,'Id']
    path = locT + str(id) + '.jpg'
    im_array=cv2.imread(path)
    im_array=resize(im_array,image_size,anti_aliasing=True)
    images.append(im_array)

X_test=np.array(images).reshape(-1,96,96,3)
del images

In [17]:
y_predict=model.predict(X_test)
y_predict=y_predict.reshape(-1,)

submission=pd.DataFrame({'Id':df_test['Id'],'Pawpularity':y_predict})
submission

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,33.814159
1,43a2262d7738e3d420d453815151079e,30.825935
2,4e429cead1848a298432a0acad014c9d,31.145409
3,80bc3ccafcc51b66303c2c263aa38486,31.730209
4,8f49844c382931444e68dffbe20228f4,34.437244
5,b03f7041962238a7c9d6537e22f9b017,33.079315
6,c978013571258ed6d4637f6e8cc9d6a3,33.725300
7,e0de453c1bffc20c22b072b34b54e50f,31.048405


In [18]:
os.chdir("/kaggle/working/")
submission.to_csv('submission.csv', index = False)